In [1]:
# trying to incorporate some ml stuff to this nba data now, too

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
lebron = pd.read_csv('data/NBA/1_lebron_james_shot_chart_1_2023.csv')
james = pd.read_csv('data/NBA/2_james_harden_shot_chart_2023.csv')
stephen = pd.read_csv('data/NBA/3_stephen_curry_shot_chart_2023.csv')

print(lebron.shape)
print(james.shape)
print(stephen.shape)

(1533, 15)
(1025, 15)
(1434, 15)


In [4]:
lebron.dtypes

top                     int64
left                    int64
date                   object
qtr                    object
time_remaining         object
result                   bool
shot_type               int64
distance_ft             int64
lead                     bool
lebron_team_score       int64
opponent_team_score     int64
opponent               object
team                   object
season                  int64
color                  object
dtype: object

In [5]:
# let's try to use the positional data from 'top' and 'left' to create a 
# model that predicts whether lebron james will make a shot or not

# first let's select the top and left columns for X_train
# and shots made for the y_train

X_train = lebron.iloc[0:1500,0:2].to_numpy().astype('int16')
y_train = lebron.iloc[0:1500,5].to_numpy().astype('int8')

X_valid = lebron.iloc[1500:1533,0:2].to_numpy().astype('int16')
y_valid = lebron.iloc[1500:1533,5].to_numpy().astype('int8')


In [6]:
# now let's scale the data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_train

array([[ 1.61134788, -0.31335703],
       [ 0.62481251,  0.29912413],
       [-0.08712023, -0.66334626],
       ...,
       [-0.20916584,  0.21162682],
       [-0.67700735,  0.78035933],
       [ 1.40793853,  1.20690871]])

In [7]:
X_train.shape

(1500, 2)

In [8]:
# time to create a model

tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[2]),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 300)               900       
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 31202 (121.88 KB)
Trainable params: 31202 (121.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [11]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    validation_data=(X_valid, y_valid))

Epoch 1/30
47/47 [==============================] - 0s 2ms/step - loss: 0.6778 - accuracy: 0.6247 - val_loss: 13.6897 - val_accuracy: 0.4545
Epoch 2/30
47/47 [==============================] - 0s 714us/step - loss: 0.6573 - accuracy: 0.6667 - val_loss: 23.5970 - val_accuracy: 0.4545
Epoch 3/30
47/47 [==============================] - 0s 677us/step - loss: 0.6461 - accuracy: 0.6613 - val_loss: 30.7747 - val_accuracy: 0.4545
Epoch 4/30
47/47 [==============================] - 0s 674us/step - loss: 0.6396 - accuracy: 0.6600 - val_loss: 36.3428 - val_accuracy: 0.4545
Epoch 5/30
47/47 [==============================] - 0s 689us/step - loss: 0.6356 - accuracy: 0.6580 - val_loss: 40.9089 - val_accuracy: 0.4545
Epoch 6/30
47/47 [==============================] - 0s 672us/step - loss: 0.6331 - accuracy: 0.6587 - val_loss: 44.2310 - val_accuracy: 0.4545
Epoch 7/30
47/47 [==============================] - 0s 648us/step - loss: 0.6315 - accuracy: 0.6567 - val_loss: 47.1228 - val_accuracy: 0.4545
E